In [1]:
import pandas as pd
import pyodbc 
import numpy as np


In [2]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj


cc=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\{0} 2007-2017.xlsx'.format('Asia'),sheet_name='Index'))

In [3]:
def fundamental_df_clean(region,year,tabs,volume_low,size_low):
    data_dict={}

    for s in tabs:
        df=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\{0} {1}.xlsx'.\
                                     format(region,year),sheet_name=s))
        data_dict[s]=df

    # select the right universe for US
    universe_evaluate=(data_dict["Market cap"]>size_low)&(data_dict["Volume"]>volume_low)
    data_dict["Revision"]=data_dict["Revision"][universe_evaluate]
    data_dict["Market cap"]=data_dict["Market cap"][data_dict["Market cap"]>size_low]    
    
    
    fundamental_df=data_dict["Date"].unstack(-1).to_frame()
    fundamental_df.index=fundamental_df.apply(lambda x: x.name[1],axis=1)
    fundamental_df.columns=["Date"]
    
    for i in tabs:
        try:
            fundamental_df[i]=data_dict[i].unstack(-1).values
        except ValueError:
            pass
        
    fundamental_df=fundamental_df.dropna(how="all",axis=0)
    fundamental_df=fundamental_df[-np.isnan(fundamental_df["Revision"])]
    fundamental_df=fundamental_df[-np.isnat(fundamental_df.Date)]
    if 'Property' in data_dict.keys():
        fundamental_df["Supersector"]=fundamental_df.apply(lambda x:data_dict["Property"].loc[x.name,"Supersector"]\
                                               if type(x.name)==str else None,axis=1)
    if 'Index' in data_dict.keys():
        fundamental_df["Index"]=fundamental_df.apply(lambda x: data_dict["Index"].loc[x.name].iloc[0] if type(x.name)==str\
                                                     else None,axis=1)
    
    fundamental_df=fundamental_df[fundamental_df.apply(lambda x: type(x.name)==str,axis=1)]
        
    return fundamental_df

In [7]:
US_2019=fundamental_df_clean('US','2019',['Date','Revision','Market cap','Volume'],3,500)

In [10]:
US_2018=fundamental_df_clean('US','2018',['Date','Revision','Market cap','Volume'],3,500)

In [9]:
US_history=fundamental_df_clean('US','2007-2017',['Date','Revision','Market cap','Volume'],3,500)

In [11]:
US=US_history.append(US_2018).append(US_2019)

In [14]:
Europe_2018=fundamental_df_clean('Europe','2018',['Date','Revision','Market cap','Volume'],3,500)

In [15]:
Europe_2019=fundamental_df_clean('Europe','2019',['Date','Revision','Market cap','Volume'],3,500)

In [16]:
Europe_history=fundamental_df_clean('Europe','2007-2017',['Date','Revision','Market cap','Volume'],3,500)

In [17]:
Europe=Europe_history.append(Europe_2018).append(Europe_2019)

In [4]:
Asia_2018=fundamental_df_clean('Asia','2018',['Date','Revision','Market cap','Volume'],3,500)

In [5]:
Asia_2019=fundamental_df_clean('Asia','2019',['Date','Revision','Market cap','Volume'],3,500)

In [6]:
Asia_history=fundamental_df_clean('Asia','2007-2017',['Date','Revision','Market cap','Volume'],3,500)

In [8]:
Asia=Asia_history.append(Asia_2018).append(Asia_2019)

In [12]:
US.to_csv('US.csv')

In [18]:
Europe.to_csv('Europe.csv')

In [9]:
Asia.to_csv('Asia.csv')